<a href="https://colab.research.google.com/github/geee05/Fine-Tuning-hugging-face-models/blob/main/RoBERTa_fine_tuning_(custom_data).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets huggingface_hub tensorboard==2.11
!sudo apt-get install git-lfs --yes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 108.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-dat

In [2]:
import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)
from huggingface_hub import HfFolder, notebook_login

In [3]:
notebook_login()

In [4]:
model_id = "roberta-base"
##dataset_id = "ag_news"
# replace the value with your model: ex <hugging-face-user>/<model-name>
repository_id = "geee05/roberta-base_text"

In [5]:
import pandas as pd
from datasets import ClassLabel, Dataset
data=pd.read_csv('/content/DAIC_train_3sp_sampled.csv')
test=pd.read_csv('/content/DAIC_test_3sp.csv')
#label_test = ClassLabel(num_classes=2, names=[0, 1])

data.rename(columns = {'Group':'label'}, inplace = True)
test.rename(columns = {'Group':'label'}, inplace = True)

dataset = Dataset.from_dict(data)
test_dataset=Dataset.from_dict(test)

dataset = dataset.class_encode_column("label")
test_dataset = test_dataset.class_encode_column("label")

Stringifying the column:   0%|          | 0/10264 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/10264 [00:00<?, ? examples/s]

Stringifying the column:   0%|          | 0/1914 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/1914 [00:00<?, ? examples/s]

In [6]:
data['label'].value_counts()

0    5132
1    5132
Name: label, dtype: int64

In [7]:
type(dataset)

datasets.arrow_dataset.Dataset

In [8]:

# Training and testing datasets
train_dataset = dataset
test_dataset = test_dataset
# Validation dataset
##val_dataset = dataset['test'].shard(num_shards=2, index=1)

# Preprocessing
tokenizer = RobertaTokenizerFast.from_pretrained(model_id)

# This function tokenizes the input text using the RoBERTa tokenizer. 
# It applies padding and truncation to ensure that all sequences have the same length (256 tokens).
def tokenize(batch):
    #print(tokenizer(batch["text"], padding=True, truncation=True, max_length=256)['input_ids'])
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=256)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
##val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

Map:   0%|          | 0/10264 [00:00<?, ? examples/s]

Map:   0%|          | 0/1914 [00:00<?, ? examples/s]

In [9]:
type(train_dataset)

datasets.arrow_dataset.Dataset

In [10]:
# Set dataset format
train_dataset.set_format("torch", columns=['input_ids','attention_mask','label'])
#val_dataset.set_format("torch", columns=['input_ids','attention_mask','label'])
test_dataset.set_format("torch", columns=['input_ids','attention_mask','label'])

In [11]:
type(train_dataset)

datasets.arrow_dataset.Dataset

In [12]:
# Extract the number of classess and their names
num_labels = dataset.features['label'].num_classes
class_names = dataset.features['label'].names
print(f"number of labels: {num_labels}")
print(f"the labels: {class_names}")

number of labels: 2
the labels: ['0', '1']


In [13]:
# Create an id2label mapping
id2label = {i: label for i, label in enumerate(class_names)}

# Update the model's configuration with the id2label mapping
config = AutoConfig.from_pretrained(model_id)
config.update({"id2label": id2label})

In [14]:
#!pip uninstall -y transformers accelerate
#!pip install transformers accelerate


  
   learning_rate=2e-5,


In [16]:
import numpy as np
# Model
model = RobertaForSequenceClassification.from_pretrained(model_id, config=config)

# TrainingArguments
training_args = TrainingArguments(
    output_dir=repository_id,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="tensorboard",
    push_to_hub=True,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)
from datasets import load_metric
def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}
    
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

Download file pytorch_model.bin:   0%|          | 1.40k/476M [00:00<?, ?B/s]

Download file logs/events.out.tfevents.1686021106.e8b23e51d18d.2136.2:  17%|#7        | 7.40k/43.1k [00:00<?, …

Download file logs/events.out.tfevents.1686125913.8814da7f2a97.3691.0:  72%|#######2  | 17.0k/23.5k [00:00<?, …

Download file logs/events.out.tfevents.1686026891.bb2f0e8c85db.1969.8:  24%|##3       | 16.5k/69.9k [00:00<?, …

...

Download file training_args.bin: 100%|##########| 3.87k/3.87k [00:00<?, ?B/s]

Clean file training_args.bin:  26%|##5       | 1.00k/3.87k [00:00<?, ?B/s]

Download file logs/events.out.tfevents.1686025779.bb2f0e8c85db.1969.2: 100%|##########| 409/409 [00:00<?, ?B/s…

Clean file logs/events.out.tfevents.1686025779.bb2f0e8c85db.1969.2: 100%|##########| 409/409 [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/476M [00:00<?, ?B/s]

In [17]:
# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.565800,0.848480,0.531348,0.422408
2,0.253800,1.288743,0.623302,0.343949
3,0.094600,2.002613,0.646813,0.325349
4,0.001000,2.443119,0.625914,0.370826
5,0.000400,2.410700,0.656740,0.280394


<ipython-input-16-4f78bb2bd6fc>:29: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


TrainOutput(global_step=6415, training_loss=0.30398638083818, metrics={'train_runtime': 2691.3567, 'train_samples_per_second': 19.068, 'train_steps_per_second': 2.384, 'total_flos': 6382210869883200.0, 'train_loss': 0.30398638083818, 'epoch': 5.0})

In [18]:
# Evaluate the model
trainer.evaluate()

{'eval_loss': 0.8484804630279541,
 'eval_accuracy': 0.5313479623824452,
 'eval_f1': 0.42240824211204125,
 'eval_runtime': 23.0491,
 'eval_samples_per_second': 83.04,
 'eval_steps_per_second': 10.413,
 'epoch': 5.0}